In [5]:
from stmpy import Machine, Driver

import ipywidgets as widgets
from IPython.display import display

from threading import Thread
import paho.mqtt.client as mqtt

broker, port = "test.mosquitto.org", 1883

class Charger:
    # MQTT
    def on_connect(self, client, userdata, flags, rc):
        print("on_connect(): {}".format(mqtt.connack_string(rc)))
    
    def on_message(self, client, userdata, msg):
        print("on_message(): topic: {}".format(msg.topic))
        if msg.message == "scu_finshed_charge": # TODO: add scu messages, make sure msg data is correct
            self.on_buttonChargingFinished_press

    # Buttons & functions
    def on_buttonConnect_press(self, b):
        self.stm.send('car_connected') # <---- here we send a signal
        self.client.publish("from_scu", 8) # should be "from_charger", but this script subscribes to "from_scu". Good for testing.
    
    def on_buttonCharge_press(self, b):
        self.stm.send('request_charging') # <---- here we send a signal
    
    def on_buttonDisconnect_press(self, b):
        self.stm.send('car_disconnected') # <---- here we send a signal

    def on_buttonChargingFinished_press(self, b):
        self.stm.send('charging_finished') # <---- here we send a signal
    
    def on_buttonChargingAborted_press(self, b):
        self.stm.send('charging_aborted') # <---- here we send a signal

    def update_scu(self):
        print("Updating SCU")
    
    def charge_car(self):
        print("Charging car")
            
    def __init__(self):
        # MQTT:
        self.client = mqtt.Client(callback_api_version=mqtt.CallbackAPIVersion.VERSION1)

        # load images and store them
        self.buttonConnect = widgets.Button(description="Connect car")
        self.buttonConnect.on_click(self.on_buttonConnect_press)
        
        self.buttonCharge = widgets.Button(description="Charge")
        self.buttonCharge.on_click(self.on_buttonCharge_press)

        self.buttonDisconnect = widgets.Button(description="Disconnect car")
        self.buttonDisconnect.on_click(self.on_buttonDisconnect_press)

        self.buttonChargingFinished = widgets.Button(description="Finish charging")
        self.buttonChargingFinished.on_click(self.on_buttonChargingFinished_press)

        self.buttonChargingAborted = widgets.Button(description="Abort charging")
        self.buttonChargingAborted.on_click(self.on_buttonChargingAborted_press)

        display(self.buttonConnect, self.buttonCharge, self.buttonDisconnect, self.buttonChargingFinished, self.buttonChargingAborted)

    def start(self):
        self.client.on_connect = self.on_connect
        self.client.on_message = self.on_message
        print("Connecting to {}:{}".format(broker, port))
        self.client.connect(broker, port)

        self.client.subscribe("from_scu")

In [6]:
charger = Charger()

t0_initial = {'source': 'initial',
      'target': 'idle'}

t1_car_connected = {'trigger': 'car_connected',
        'source': 'idle',
        'target': 'connected'
}

t2_request_charging = {'trigger': 'request_charging',
        'source': 'connected',
        'target': 'charging'
}
t3_car_disconnected = {'trigger': 'car_disconnected',
        'source': 'connected',
        'target': 'idle'
}
t4_charging_finished = {'trigger': 'charging_finished',
        'source': 'charging',
        'target': 'connected'
}
t5_charging_aborted = {'trigger': 'charging_aborted',
        'source': 'charging',
        'target': 'connected'
}

idle = {'name': 'idle'}

connected = {'name': 'connected',
             'entry': 'update_scu'}

charging = {'name': 'charging',
            'entry': 'charge_car'}

machine = Machine(name='charger', transitions=[t0_initial, t1_car_connected, t2_request_charging, t3_car_disconnected, t4_charging_finished, t5_charging_aborted],
                                states=[idle, connected, charging], obj=charger)

charger.stm = machine
driver = Driver()
driver.add_machine(machine)
driver.start()

C:\Users\afwol\AppData\Local\Temp/ipykernel_23204/2169089265.py:46: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  self.client = mqtt.Client(callback_api_version=mqtt.CallbackAPIVersion.VERSION1)


Button(description='Connect car', style=ButtonStyle())

Button(description='Charge', style=ButtonStyle())

Button(description='Disconnect car', style=ButtonStyle())

Button(description='Finish charging', style=ButtonStyle())

...

Updating SCU
Charging car
Updating SCU
Charging car


Machine charger is in state charging and received event car_disconnected, but no transition with this event is declared!


Updating SCU


Machine charger is in state idle and received event car_disconnected, but no transition with this event is declared!


Updating SCU
